# Рекомедации на основе содержания

датасет MovieLens на 100к


In [ ]:
# подгружаем библиотеки

import pandas as pd
import numpy as np
import math
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# загружаем наборы данных

links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

In [ ]:
# создадим функцию для обучения модели линейной регрессии и расчета метрик RMSE и R2

def lin_reg(X,y):

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # разбиение выборок
    model = LinearRegression()
    model.fit(X_train, y_train)                                                               # обучение модели

    y_train_pred = model.predict(X_train)                 # предсказание на train
    y_test_pred = model.predict(X_test)                   # предсказание на test

    mse_train = mean_squared_error(y_train, y_train_pred) # среднеквадратическая ошибка для обучающей выборки
    mse_test = mean_squared_error(y_test, y_test_pred)    # среднеквадратическая ошибка для тестовой выборки

    rmse_train = math.sqrt(mse_train)                     # корень из среднеквадратической ошибки для обучающей выборки
    rmse_test = math.sqrt(mse_test)                       # корень из среднеквадратической ошибки для тестовой выборки

    r2_train = r2_score(y_train, y_train_pred)            # коэффициент детерминации для обучающей выборки
    r2_test = r2_score(y_test, y_test_pred)               # коэффициент детерминации для тестовой выборки

    print('RMSE train: {:.2f}, test: {:.2f}'.format(rmse_train, rmse_test))
    print('R2 train: {:.4f}, test: {:.4f}'.format(r2_train, r2_test))

In [ ]:
# объединим датасеты movies и tags, чтобы в одном датафрейме были и жанры, и теги

movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [ ]:
# соединяем методом inner (так как с outer join образуются NaN, следовательно, в модель подавать будет невозможно)

df_movies = movies.merge(tags, on='movieId')
df_movies.head()

,movieId,title,genres,userId,tag,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336,pixar,1139045764
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,474,pixar,1137206825
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,567,fun,1525286013
3,2,Jumanji (1995),Adventure|Children|Fantasy,62,fantasy,1528843929
4,2,Jumanji (1995),Adventure|Children|Fantasy,62,magic board game,1528843932


In [ ]:
df_movies.shape

(3683, 6)

In [ ]:
df_movies.isna().sum()

movieId      0
title        0
genres       0
userId       0
tag          0
timestamp    0
dtype: int64

In [ ]:
# соединим полученный датафрейм с датасетом ratings, тоже методом inner
# по ключам movieId и userId

df = df_movies.merge(ratings, how='inner', on=['movieId', 'userId'])
df.head()

,movieId,title,genres,userId,tag,timestamp_x,rating,timestamp_y
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336,pixar,1139045764,4.0,1122227329
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,474,pixar,1137206825,4.0,978575760
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,567,fun,1525286013,3.5,1525286001
3,2,Jumanji (1995),Adventure|Children|Fantasy,62,fantasy,1528843929,4.0,1528843890
4,2,Jumanji (1995),Adventure|Children|Fantasy,62,magic board game,1528843932,4.0,1528843890


In [ ]:
del df['timestamp_x']
del df['timestamp_y']

In [ ]:
df.head()

,movieId,title,genres,userId,tag,rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336,pixar,4.0
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,474,pixar,4.0
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,567,fun,3.5
3,2,Jumanji (1995),Adventure|Children|Fantasy,62,fantasy,4.0
4,2,Jumanji (1995),Adventure|Children|Fantasy,62,magic board game,4.0


In [ ]:
# из ста тысяч оценок в объединенном датафрейме осталось лишь три с половиной

df.shape

(3476, 6)

# tf-idf

In [ ]:
# переведем теги и жанры в векторы

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['tag'] + "" + df['genres'])
tfidf_matrix

<3476x2634 sparse matrix of type '<class 'numpy.float64'>'
	with 11835 stored elements in Compressed Sparse Row format>

In [ ]:
# фичи и таргет
X = tfidf_matrix
y = df['rating']

In [ ]:
# модель линейной регрессии

lin_reg(X,y)

RMSE train: 0.41, test: 0.85
R2 train: 0.7706, test: 0.0048


Видим, что ошибка на тесте чуть лучше наивной модели (т.е. предсказание среднего рейтинга по датасету).

Возможно, модель плохо обучилась, так как переобучилась: считается, что линейная регрессия плохо работает с разреженными матрицами, и, возможно, нужно было использовать Elastic Net.

Кроме того, очевидна и плохая предобработка тегов и жанров.

# Расчет статистик mean, median, variance по пользователю и по фильму

In [ ]:
df.head()

,movieId,title,genres,userId,tag,rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336,pixar,4.0
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,474,pixar,4.0
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,567,fun,3.5
3,2,Jumanji (1995),Adventure|Children|Fantasy,62,fantasy,4.0
4,2,Jumanji (1995),Adventure|Children|Fantasy,62,magic board game,4.0


In [ ]:
# создадим столбцы со средним значением, медианой и дисперсией для пользователя и для фильма

df['user_mean'] = df.groupby('userId')['rating'].transform('mean')

In [ ]:
df['user_median'] = df.groupby('userId')['rating'].transform('median')


In [ ]:
df['user_variance'] = df.groupby('userId')['rating'].transform('var')


In [ ]:
df['movie_mean'] = df.groupby('movieId')['rating'].transform('mean')


In [ ]:
df['movie_median'] = df.groupby('movieId')['rating'].transform('median')


In [ ]:
df['movie_variance'] = df.groupby('movieId')['rating'].transform('var')


In [ ]:
df.head()

,movieId,title,genres,userId,tag,rating,user_mean,user_median,user_variance,movie_mean,movie_median,movie_variance
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336,pixar,4.0,3.777778,4.0,0.444444,3.833333,4.0,0.083333
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,474,pixar,4.0,3.701909,4.0,0.666033,3.833333,4.0,0.083333
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,567,fun,3.5,3.917824,4.0,0.649264,3.833333,4.0,0.083333
3,2,Jumanji (1995),Adventure|Children|Fantasy,62,fantasy,4.0,3.937838,4.0,0.477155,3.750000,4.0,0.250000
4,2,Jumanji (1995),Adventure|Children|Fantasy,62,magic board game,4.0,3.937838,4.0,0.477155,3.750000,4.0,0.250000


In [ ]:
# есть пропуски в дисперсии рейтинга фильма, если только один пользователь оценил фильм
# также есть пропуски в дисперсии рейтингов пользователя, если пользоваетель оценил только один фильм

df.isna().sum()

movieId             0
title               0
genres              0
userId              0
tag                 0
rating              0
user_mean           0
user_median         0
user_variance       6
movie_mean          0
movie_median        0
movie_variance    933
dtype: int64

In [ ]:
# заполняем пропуски в дисперсиях рейтинга фильма единственным указанным рейтингом

df.movie_variance.fillna(df.rating, inplace=True)
df.user_variance.fillna(df.rating, inplace=True)

In [ ]:
df.isna().sum()

movieId           0
title             0
genres            0
userId            0
tag               0
rating            0
user_mean         0
user_median       0
user_variance     0
movie_mean        0
movie_median      0
movie_variance    0
dtype: int64

# Регрессия без tf-idf


In [ ]:
# фичи и таргет

X = df[['userId', 'movieId', 'user_mean', 'user_median', 'user_variance', 'movie_mean', 'movie_median', 'movie_variance']]
y = df['rating']

In [ ]:
# модель
lin_reg(X,y)

RMSE train: 0.25, test: 0.25
R2 train: 0.9124, test: 0.9133


Обучаясь на статистиках по пользователю и фильму, модель достигла высокого коэффициента детерминации, в том числе на тестовой выборке: 0,91. Ошибка RMSE и на обучающей, и на тестой выборках небольшая: 0,25.